실행 위치: 로컬

In [ ]:
# PYTHONPATH 설정
import sys, os 

BASE_DIR = os.getcwd()
for _ in range(1):
    BASE_DIR = os.path.dirname(BASE_DIR)

sys.path.insert(0, BASE_DIR)
print(sys.path)

# 1. Tool 함수 정의

In [ ]:
def get_current_weather(city: str, unit: "str"):
    return (
        f"The weather in {city},is 85 degrees {unit}. It is "
        "partly cloudly, with highs in the 90's."
    )


tool_functions = {"get_current_weather": get_current_weather}

In [ ]:
tool_description = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "주어진 city에 대한 현재 온도를 리턴",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "온도를 파악하고자 하는 도시, 예시) '서울'",
                    },
                    "unit": {
                        "type": "string",
                        "description": "온도를 표시할 단위",
                        "enum": ["섭씨", "화씨"],
                    },
                },
                "required": ["city", "unit"],
            },
        },
    }
]

# 2. 추론 수행 (툴 제공)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "현재 강릉의 날씨는 몇도인지 알려줘"
    }
]




In [ ]:
from fastapi_app.app.core.config import settings

vllm_ip_port = settings.VLLM_IP_PORT.rstrip("/")
vllm_api_key = settings.VLLM_API_KEY
vllm_chat_completions_url = f"{vllm_ip_port}/v1/chat/completions"

payload = {
    "model": "openai/gpt-oss-20b",
    "messages": messages,
    "tools": tool_description,
    "tool_choice": "auto",  # 모델이 스스로 판단하여 툴 사용 여부 결정
    "temperature": 0
}

headers={"Content-Type": "application/json", "Authorization": f"Bearer {vllm_api_key}"}

In [ ]:
import httpx 

async with httpx.AsyncClient(timeout=30.0) as client:
    response = await client.post(url=vllm_chat_completions_url, json=payload, headers=headers)

# 3. 응답 확인

## 3-1. vLLM 모델 기본 로드

In [ ]:
# 첫 번째 시도
import json

if response.status_code == 200:
    model_response = response.json()
    print(json.dumps(model_response, indent=4, ensure_ascii=False))
    
else:
    print(f"에러 발생: {response.status_code}")
    print(response.text)

- vLLM 모델 기본 로드시 모델의 Tool에 대한 응답이 파싱하기 어려운 형태로 출력

## 3-2. vLLM tool-call-parser 지정

In [ ]:
# vLLM 모델 로드시 tool-call-parser=openai 지정 후 시도

if response.status_code == 200:
    model_response = response.json()
    print(json.dumps(model_response, indent=4, ensure_ascii=False))
    
else:
    print(f"에러 발생: {response.status_code}")
    print(response.text)

### 툴 실행(모델이 전달한 tool 내용을 바탕으로 실행)

In [ ]:
# 모델이 응답한 내용을 바탕으로 함수명과 변수를 추출

functions = model_response["choices"][0]["message"]["tool_calls"]
for i, function in enumerate(functions):
    print(f"{i} 번째 함수")
    function_name = function["function"]["name"]
    args = json.loads(function["function"]["arguments"])

    print(f"함수명: {function_name}")
    print(f"변수: {args}")


In [ ]:
# 함수 실행
tool_exec_rslt = tool_functions[function_name](**args)
print(tool_exec_rslt)

In [ ]:
# 모델의 응답과 툴 실행 결과를 messages에 담아 요청

model_message = model_response["choices"][0]["message"]
tool_message = {"role": "tool", "content": tool_exec_rslt}

messages.append(model_message)
messages.append(tool_message)

In [ ]:
messages

# 4. 툴 실행 결과를 모델에게 전달

In [ ]:
import httpx 

async with httpx.AsyncClient(timeout=60.0) as client:
    response = await client.post(url=vllm_chat_completions_url, json=payload, headers=headers)

In [ ]:

import json

if response.status_code == 200:
    model_response = response.json()
    print(json.dumps(model_response, indent=4, ensure_ascii=False))
    
else:
    print(f"에러 발생: {response.status_code}")
    print(response.text)